<p align="center">
  <img src="ucl_banner.avif" alt="UEFA Champions League" width="100%"/>
</p>


# Visualització interactiva de la UEFA Champions League (1955–2016)

## Evolució històrica, rendiment dels clubs i patrons competitius

**Assignatura:** Visualització de dades  
**Autor:** Ot De la Varga Iborra  
**Data:** 13 de gener de 2026


## 1. Introducció

La UEFA Champions League és una de les competicions esportives més prestigioses del futbol europeu i mundial. Des de la seva creació l’any 1955, la competició ha evolucionat tant en format com en dimensió, reflectint canvis estructurals, econòmics i esportius dins del futbol professional.

Aquest projecte té com a objectiu explorar i comunicar l’evolució històrica de la competició i els seus clubs a través d’una serie de visualitzacions interactives basades en dades de partits i resultats entre els anys 1955 i 2016. Mitjançant tècniques de visualització de dades, es busca identificar patrons, tendències i fites rellevants, així com facilitar la comparació entre clubs, temporades i etapes de la competició.

La visualització està pensada tant per a un públic no especialitzat, interessat en la història del futbol europeu, com per a usuaris amb un perfil més analític que vulguin explorar el rendiment dels clubs al llarg del temps.


In [1]:
import pandas as pd
import numpy as np
import re
import pycountry

from itables import show

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
import plotly.colors as pc

import plotly.io as pio



In [2]:
pio.renderers.default = 'notebook_connected'

In [3]:
# Estil general per visuals estàtiques
sns.set_theme(style="whitegrid", context="talk")
plt.rcParams["figure.figsize"] = (12, 6)

# 2. Datasets

En aquesta pràctica s'ha fet ús de 3 conjunts de dades diferents:

- *dataset_final.csv*: mostra tots els partits de la Champions entre els anys 1955 i 2016.
- *AllTimeRankingByClub.csv*: mostra un ranking de tots els equips participants.
- *UCL_Finals_1955-2016.csv*: mostra totes les finals dutes a terme entre el 1955 i 2016.

A continuació es pot veure el contingut dels datasets.

In [4]:
# 1. Càrrega i exploració de dades
matches = pd.read_csv("data/dataset_final.csv")
ranking = pd.read_csv("data/AllTimeRankingByClub.csv", encoding="utf-16")
finals = pd.read_csv("data/UCL_Finals_1955-2016.csv")

In [5]:
show(matches.fillna('-'))

Loading ITables v2.6.2 from the internet... (need help?)


In [6]:
show(ranking.fillna('-'))

Loading ITables v2.6.2 from the internet... (need help?)


In [7]:
show(finals.fillna('-'))

Loading ITables v2.6.2 from the internet... (need help?)


In [8]:
# fig9 = go.Figure(data=[go.Table(
#     header=dict(values=list(matches.columns)),
#     cells=dict(values=[matches[col] for col in matches.columns])
# )])

# fig9.update_layout(
#     title="Dataset Partits"
# )

# fig9.show()


In [9]:
# Obtenció del nom dels equps
def clean_team_name(team):
    return team.split("›")[0].strip()

In [10]:
matches["Team 1"] = matches["Team 1"].apply(clean_team_name)
matches["Team 2"] = matches["Team 2"].apply(clean_team_name)


In [11]:
# Obtenció dels gols de local i visitant
def extract_goals(result, idx):
    if pd.isna(result):
        return np.nan
    try:
        return int(result.split("-")[idx])
    except:
        return np.nan

In [12]:
matches["HomeGoals"] = matches["FT"].apply(lambda x: extract_goals(x, 0))
matches["AwayGoals"] = matches["FT"].apply(lambda x: extract_goals(x, 1))
matches["HomeGoals"] = matches["HomeGoals"].astype("Int64")
matches["AwayGoals"] = matches["AwayGoals"].astype("Int64")


In [13]:
# Temporada
matches['Season'] = matches['source_folder']

In [14]:
home = matches[["Season", "Team 1", "HomeGoals", "AwayGoals"]].copy()
home.columns = ["Season", "Team", "GoalsFor", "GoalsAgainst"]
home["Win"] = home["GoalsFor"] > home["GoalsAgainst"]

away = matches[["Season", "Team 2", "AwayGoals", "HomeGoals"]].copy()
away.columns = ["Season", "Team", "GoalsFor", "GoalsAgainst"]
away["Win"] = away["GoalsFor"] > away["GoalsAgainst"]

teams = pd.concat([home, away], ignore_index=True)


In [15]:
season_team = (
    teams
    .groupby(["Season", "Team"])
    .agg(
        matches_played=("Team", "count"),
        goals_for=("GoalsFor", "sum"),
        goals_against=("GoalsAgainst", "sum"),
        wins=("Win", "sum")
    )
    .reset_index()
)

season_team["win_rate"] = season_team["wins"] / season_team["matches_played"]


# 3. Visualitzacions

### Equips dominants per temporada

Aquesta visualització mostra, per a cada temporada, quin equip ha aconseguit el major nombre de victòries. L’objectiu és identificar períodes de domini esportiu i observar com aquest lideratge varia al llarg del temps.

La visualització permet detectar dinasties clares, així com temporades en què la competició ha estat més equilibrada. També ajuda a contextualitzar l’impacte dels canvis en el format del torneig, especialment a partir de la introducció de la fase de grups.


In [16]:
season_team['Year'] = season_team['Season'].str.split('-').str[1].apply(lambda x: '20' + x if len(x) == 2 and int(x) < 50 else '19' + x if len(x) == 2 else x).astype(int)

In [17]:
top_per_season = (
    season_team
    .dropna(subset=["wins"])              # eliminem temporades sense dades
    .sort_values(["Year", "wins"], ascending=[True, False])
    .groupby("Year")
    .head(1)
)


In [18]:
# Assegura’t que 'Year' és string per tractar-lo com a categòric
top_per_season['Year'] = top_per_season['Year'].astype(str)

fig1 = px.bar(
    top_per_season,
    x='Year',
    y='wins',
    color='Team',
    color_discrete_sequence=px.colors.qualitative.Alphabet,
    title="Equips dominants per temporada (Equip amb més victories)",
    labels={'wins':'Victòries', 'Year':'Temporada'}
)

# Forçar l'ordre de l'eix X segons els anys originals
fig1.update_xaxes(
    type='category',
    categoryorder='array',
    categoryarray=top_per_season['Year'].sort_values().tolist(),  # ordre cronològic
    tickangle=-45
)

fig1.update_layout(
    legend_title_text='Equip',
    margin=dict(b=150)
)

fig1.show()



Com es pot observar el nombre màxim de victòries es manté bastant constant durant els anys fins l'any 1993 on la competició va passar d'anomenar-se Copa d'Europa a *UEFA Champions League*. Predominen els equips que més cops han guanyat la competició com el FC Barcelona, el Bayern de Munich, l'AC Milan, etc.

In [19]:
# Normalitzar Season a finals
finals['Season'] = finals['Season'].str.replace('–', '-', regex=False)

# (Opcional però recomanat) assegurar tipus string
top_per_season['Season'] = top_per_season['Season'].astype(str)
finals['Season'] = finals['Season'].astype(str)

# Fer el merge
top_per_season = top_per_season.merge(
    finals[['Season', 'Winners']],
    on='Season',
    how='left'
)



In [20]:
top_per_season['Team_norm'] = (
    top_per_season['Team']
    .str.lower()
    .str.strip()
)

top_per_season['Winners_norm'] = (
    top_per_season['Winners']
    .str.lower()
    .str.strip()
)

In [21]:
top_per_season['is_winner_team'] = top_per_season.apply(
    lambda row: row['Winners_norm'] in row['Team_norm']
    if pd.notna(row['Winners_norm']) else False,
    axis=1
)

In [22]:
count_winner_matches = top_per_season['is_winner_team'].sum()

### Comparació entre l’equip amb més victòries i l’equip guanyador

Aquesta visualització compara, per temporada, l’equip amb més victòries totals amb l’equip que finalment es proclama campió de la competició.

La comparació posa de manifest que acumular més victòries no sempre garanteix guanyar el torneig, especialment en competicions d’eliminació directa. Aquest contrast permet analitzar l’eficiència competitiva dels equips i destacar temporades amb resultats inesperats.

Quan l'equip amb més victòries guanya la competició la linia es mostra en verd, en cas contrari en vermell.

In [23]:
# Colors per fila segons si és campió o no
row_colors = [
    '#d4edda' if is_winner else '#f8d7da'
    for is_winner in top_per_season['is_winner_team']
]

fig2 = go.Figure(data=[go.Table(
    header=dict(
        values=['Season', 'Team', 'Winners'],
        fill_color='lightgrey',
        align='left'
    ),
    cells=dict(
        values=[
            top_per_season['Season'],
            top_per_season['Team'],
            top_per_season['Winners']
        ],
        fill_color=[row_colors, row_colors, row_colors],  # una llista per columna
        align='left'
    )
)])

fig2.update_layout(
    title='Equips vs Campions per Temporada',
    height=600
)

fig2.show()

total_rows = len(top_per_season)
true_rows = top_per_season['is_winner_team'].sum()  # True = 1, False = 0

print(f"{true_rows} de {total_rows} files són campions ({true_rows/total_rows:.1%})")



33 de 61 files són campions (54.1%)


El principal aspecte que destaca és que de les 61 temporades mostrades, en 33 l'equip amb més victòries ha estat el guanyador, poc més d'un 50% (54,1%).

També es poden observar diferents perídoes on l'equip que més guanya acaba guanyant el torneig, per exemple de la temporada 2005-06 a la 2015-16, només en 2 casos no s'ha complert; i períodes on passa tot el contrari com de la temporada 1989-90 a la 1995-96, 7 anys seguits on el guanyador no era l'equip més dominant.

In [24]:
# Crear una columna amb els equips ordenats per fila (per ignorar local/visitant)
matches['matchup'] = matches.apply(
    lambda row: tuple(sorted([row['Team 1'], row['Team 2']])),
    axis=1
)

# Comptar quants cops es repeteix cada matchup
top_matches = matches['matchup'].value_counts().reset_index()
top_matches.columns = ['matchup', 'count']

# Mostrar només el top 10
top_10 = top_matches.head(10)

### Partits més repetits en la història de la competició

Aquesta visualització recull els enfrontaments entre equips que s’han produït amb més freqüència al llarg de la història de la UEFA Champions League.

Els resultats reflecteixen rivalitats recurrents i la presència continuada de determinats clubs en fases avançades del torneig. Això permet identificar patrons estructurals i entendre quins equips han estat habituals en l’elit europea durant dècades.


In [25]:
# -------------------------
# 1️⃣ Crear columna unordered per comptar partits sense importar local/visitant
matches['matchup'] = matches.apply(
    lambda row: tuple(sorted([row['Team 1'], row['Team 2']])),
    axis=1
)

# -------------------------
# 2️⃣ Comptar repeticions de cada matchup
match_counts = matches['matchup'].value_counts().reset_index()
match_counts.columns = ['matchup', 'count']

# -------------------------
# 3️⃣ Afegir percentatge respecte al total de partits
total_matches = len(matches)
match_counts['percent'] = match_counts['count'] / total_matches * 100

# -------------------------
# 4️⃣ Top 10
top_10 = match_counts.head(10).copy()  # copiar per evitar warnings

# Separar tuple per visualització
top_10['teamA'] = top_10['matchup'].apply(lambda x: x[0])
top_10['teamB'] = top_10['matchup'].apply(lambda x: x[1])
top_10['matchup_label'] = top_10['teamA'] + " vs " + top_10['teamB']

# -------------------------
# 5️⃣ Gràfic interactiu
fig = px.bar(
    top_10,
    x='matchup_label',
    y='count',
    text='count',  # mostrar número de repeticions dins de la barra
    hover_data={'percent': ':.1f'},  # percentatge al hover
    title='Top 10 partits més repetits de la història',
    labels={'count': 'Repeticions', 'matchup_label': 'Partit', 'percent': '% del total de partits'}
)

fig.update_layout(
    xaxis_tickangle=-45
)

# -------------------------
# 6️⃣ Mostrar gràfic
fig.show()


Com es pot veure, hi ha una correlació entre les temporades participades i els enfrontaments més repetits ja que el partit més repetit de la història s'ha dut a terme entre el top 2 equips del rànking històric, el Bayern de Múnich i el Real Madrid.

La resta de partits repetits es duu a terme entre equips del top 10 del rànking històric.

### Evolució de trofeus al llarg dels anys

Aquesta visualització mostra com s’han distribuït els títols de la Champions League al llarg del temps entre els diferents clubs.

Permet observar tant la concentració de trofeus en uns pocs equips com l’aparició puntual de nous guanyadors. La representació temporal facilita la identificació de períodes de domini i de canvis en l’hegemonia europea.

Destaca el domini inicial del Real Madrid guanyant-ne 5 de seguides però també molts anys sense ser campió com de 1966 a 1998 (32 anys). També equips com el FC Barcelona que han estat molt dominants els últims 30 anys i equips com l'AC Milan que s'han coronat campions cada pocs anys (amb algunes excepcions).

In [26]:
# ---- 1️⃣ Normalitzar noms dels guanyadors ----
top_per_season['Winners'] = top_per_season['Winners'].str.strip()

# ---- 2️⃣ Comptar trofeus per any i equip ----
wins_per_year = top_per_season.groupby(['Year', 'Winners']).size().reset_index(name='wins')

# Pivotar per tenir equips com a columnes
wins_pivot = wins_per_year.pivot(index='Year', columns='Winners', values='wins').fillna(0)

# Trofeus acumulatius per any
wins_cumsum = wins_pivot.cumsum().reset_index()

# ---- 3️⃣ Escollir només els equips més exitosos per llegibilitat ----
# Excloure la columna Year abans de ordenar
team_columns = wins_cumsum.columns.drop('Year')

# Top 10 equips amb més trofeus totals
top_teams = wins_cumsum[team_columns].iloc[-1].sort_values(ascending=False).head(10).index.tolist()

# ---- 4️⃣ Crear gràfic interactiu ----

fig3 = px.line(
    wins_cumsum,
    x=wins_cumsum['Year'].astype(str),  # convertir a string per mostrar anys reals
    y=top_teams,
    labels={'value':'Trofeus', 'variable':'Equip', 'x':'Any'},
    title='Trofeus totals guanyats pels equips campions al llarg dels anys'
)

# Millorar llegenda i línies
fig3.update_layout(
    xaxis_tickangle=-45,
    hovermode='closest'
)

fig3.show()


### Resultats més repetits

Aquesta visualització analitza els marcadors finals que s’han repetit amb més freqüència en els partits de la competició.

L’objectiu és identificar patrons en els resultats, com ara la prevalença de victòries ajustades o empats, i obtenir una visió general del nivell de competitivitat històric del torneig.


In [27]:
# ---- 1️⃣ Netejar la columna FT ----
# Treure símbols que no siguin nombres o guions
matches['FT_clean'] = matches['FT'].apply(lambda x: re.sub(r'[^0-9\-]', '', str(x)))

# ---- 2️⃣ Comptar resultats ----
result_counts = matches['FT_clean'].value_counts().reset_index()
result_counts.columns = ['Resultat', 'Count']

# Opcional: top 20 més comuns
# top_results = result_counts.head(20)

# Calcular percentatge respecte al total
result_counts['Percent'] = result_counts['Count'] / result_counts['Count'].sum() * 100

# ---- 3️⃣ Treemap interactiu ----
fig4 = px.treemap(
    result_counts,
    path=['Resultat'],
    values='Count',
    color='Count',
    color_continuous_scale='Viridis',
    hover_data={'Resultat': True, 'Count': True, 'Percent': ':.1f'},
    title='Top resultats més repetits dels partits',
    width=1200,   # amplada
    height=800    # alçada
)

# Hover net: només mostrar Resultat i nombre + percentatge
fig4.update_traces(
    hovertemplate='Resultat: %{label}<br>Repeticions: %{value}<br>Percentatge: %{customdata[2]:.1f}%'
)

fig4.show()


Resultats amb pocs gols com 1-0 o 1-1 predominen ja que conjuntament formen un 20% de tots els partits jugats. Mentre que resultats més abultats amb més de 4 gols per partit són els que menys s'han dut a terme.

Un altre punt interessant és veure com les victòries més repetides són del local. Per exemple, hi ha 437 partits amb 0-1 i 716 amb 1-0, casi el doble de partits guanyats per un final de 1 gol a zero són per l'equip local. Aquest fet deixa en evidència que jugar a casa dona una influència positiva per l'equip local i negativa pel visitant. 

### Distribució geogràfica de les finals per país

Aquest mapa mostra els països dels quals els seus equips han arribat a més finals de la UEFA Champions League.

La visualització permet analitzar els països més dominants al llarg de la història i observar paísos que han tingut una única participació.


In [28]:
# ---- 1️⃣ Crear DataFrame únic amb Country i Team ----
winners_df = finals[['Country', 'Winners']].rename(columns={'Winners':'Team'})
losers_df = finals[['Country.1', 'Runners-up']].rename(columns={'Country.1':'Country','Runners-up':'Team'})

all_countries = pd.concat([winners_df, losers_df], ignore_index=True)

# Mapar Regne Unit
country_map = {
    'England': 'United Kingdom',
    'Scotland': 'United Kingdom',
    'Wales': 'United Kingdom',
    'Northern Ireland': 'United Kingdom',
}
all_countries['Country'] = all_countries['Country'].replace(country_map)

# ---- 2️⃣ Agrupar equips per país i comptar finals ----
teams_info = all_countries.groupby('Country')['Team'].apply(
    lambda x: ', '.join([f"{team} ({v})" for team, v in x.value_counts().items()])
).reset_index()
teams_info.columns = ['Country', 'Teams']

# ---- 3️⃣ Comptar finals totals per país ----
finals_count = all_countries['Country'].value_counts().reset_index()
finals_count.columns = ['Country', 'Finals_count']

# Merge info
country_agg = pd.merge(finals_count, teams_info, on='Country')

# ---- 4️⃣ Convertir noms a ISO3 ----
def country_to_iso3(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except:
        return None

country_agg['ISO3'] = country_agg['Country'].apply(country_to_iso3)
country_agg = country_agg.dropna(subset=['ISO3'])

# ---- 5️⃣ Crear mapa ----
fig5 = px.choropleth(
    country_agg,
    locations='ISO3',
    color='Finals_count',
    color_continuous_scale='Inferno',
    labels={'Finals_count':'Finals'},
    title='Número de finals per país',
    width=1400,
    height=900,
    hover_data={'Teams':True, 'Finals_count':True, 'ISO3':False}
)

# Hover amb equips i finals
fig5.update_traces(
    hovertemplate='País: %{location}<br>Finals: %{customdata[1]}<br>Equips: %{customdata[0]}'
)

fig5.update_layout(
    geo=dict(
        scope='europe',
        showframe=False,       # treu el marc
        showcoastlines=True,
        fitbounds="locations"  # centra i escala el mapa segons les localitzacions
    ),
    margin=dict(l=0, r=0, t=50, b=0)  # marges mínims, només deixa títol
)

fig5.show()


Es pot observar de forma clara que Espanya i Itàlia són els països amb més participacions a finals amb 27 però el Regne Unit és el país del qual més equips diferents hi han arribat amb 8.

### Estadis amb més finals disputades

Aquesta visualització presenta els estadis que han acollit un major nombre de finals de la Champions League.

Serveix per identificar infraestructures emblemàtiques dins del futbol europeu i entendre quins estadis han estat considerats referents en termes de capacitat, prestigi i ubicació.


In [29]:
# Comptar finals per estadi
venue_counts = finals['Venue'].value_counts().reset_index()
venue_counts.columns = ['Venue', 'Finals_count']

# Gràfic de barres (top 10)
import plotly.express as px

fig6 = px.bar(
    venue_counts.head(10),
    x='Finals_count',
    y='Venue',
    orientation='h',
    text='Finals_count',
    title='Top 10 estadis amb més finals',
    labels={
        'Finals_count': 'Finals',
        'Venue': 'Estadi'
    }
)
fig6.update_traces(textposition='outside')
fig6.update_layout(yaxis={'categoryorder':'total ascending'})
fig6.show()


### Assistència mitjana per estadi

Aquesta visualització mostra l’assistència mitjana registrada en els diferents estadis que han acollit finals (mínim 2).

L’anàlisi permet comparar la capacitat d’atracció dels estadis i contextualitzar l’impacte de factors com la mida de l’estadi, la ubicació o l’època en què s’ha disputat la final.


In [30]:
# ---- Netejar columna Attendance ----
finals['Attendance_clean'] = finals['Attend­ance'].str.replace(',', '').astype(float)

# ---- Comptar finals per estadi ----
venue_counts = finals['Venue'].value_counts().reset_index()
venue_counts.columns = ['Venue', 'Finals_count']

# ---- Mitjana d'assistència només per estadis amb ≥ 2 finals ----
venue_attendance = finals.groupby('Venue').agg(
    Finals_count=('Season','count'),
    Avg_Attendance=('Attendance_clean','mean')
).reset_index()

# Filtrar mínim 2 finals
venue_attendance = venue_attendance[venue_attendance['Finals_count'] >= 2]

# Ordenar per mitjana d'assistència descendent
venue_attendance = venue_attendance.sort_values(by='Avg_Attendance', ascending=False)

# ---- Gràfic top 10 ----
fig7 = px.bar(
    venue_attendance.head(10),
    x='Avg_Attendance',
    y='Venue',
    orientation='h',
    text='Avg_Attendance',
    title='Top 10 estadis amb més assistència mitjana (mínim 2 finals)',
    labels={
        'Avg_Attendance': 'Assistència Mitjana',
        'Venue': 'Estadi'
    }
)
fig7.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
fig7.update_layout(yaxis={'categoryorder':'total ascending'})
fig7.show()


No és cap sorpresa que camps emblemàtics com el Camp Nou (Barcelona) o Wembley (Londres) formin part del top 3.

### Finals amb més assistència

Aquesta visualització destaca les finals amb major nombre d’espectadors registrats.

Permet identificar esdeveniments especialment rellevants des del punt de vista històric i social, així com analitzar la relació entre estadi, any i afluència de públic.


In [31]:
# ---- Crear columna amb format "Winner - Runner-up" ----
top_attendance = finals[['Season','Venue','Attendance_clean','Winners','Runners-up']].sort_values(
    by='Attendance_clean', ascending=False
).head(10)
top_attendance['Matchup'] = top_attendance['Winners'] + ' - ' + top_attendance['Runners-up']

# ---- Gràfic ----
fig8 = px.bar(
    top_attendance,
    x='Attendance_clean',
    y='Venue',
    orientation='h',
    text='Attendance_clean',
    color='Winners',
    title='Top 10 finals amb més assistència',
    hover_data={'Matchup':True, 'Attendance_clean':':,.0f', 'Venue':False, 'Winners':False, 'Runners-up':False},
    labels={
        'Attendance_clean': 'Assistència',
        'Venue': 'Estadi',
        'Winners': 'Guanyador',
        'Matchup': 'Final'
    }
)

fig8.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
fig8.update_layout(yaxis={'categoryorder':'total ascending'})
fig8.show()


En aquesta gràfica es pot observa com les finals que han tingut més assistència al llarg del anys s'han dut a terme en estadis on la mitjana d'assitència és més elevada.

Afegir que les finals que superen els 100000 espectadors es van dur a terme als inicis de la competició on els estadis i els controls d'accessos eren molt menys restrictius que actualment.

In [32]:
# Llista de figures i noms de fitxer
figures = [
    (fig1, "top_victories.html"),
    (fig2, "victories_vs_guanyador.html"),
    (fig3, "partits.html"),
    (fig4, "trofeus.html"),
    (fig5, "mapa.html"),
    (fig6, "finals_estadis.html"),
    (fig7, "assistencia.html"),
    (fig8, "finals_assistencia.html")
]

for fig, filename in figures:
    fig.write_html(filename, include_plotlyjs='cdn', full_html=True)
    # print(f"Exportat: {filename}")


## Conclusions

L’anàlisi i la visualització de les dades històriques de la UEFA Champions League permeten extreure diverses conclusions rellevants sobre la dinàmica competitiva de la competició:

- El domini d’una temporada, mesurat pel nombre de victòries acumulades, **no garanteix necessàriament la consecució del títol**, fet que posa de manifest la naturalesa eliminatòria i imprevisible del torneig.
- Els enfrontaments més repetits al llarg dels anys es produeixen principalment entre **equips amb una elevada presència continuada a la competició**, reflectint la concentració d’elit esportiva.
- S’identifiquen **períodes clars de dominació** per part de determinats clubs, que aconsegueixen guanyar la competició de manera recurrent en etapes concretes.
- La presència en finals està fortament dominada per equips pertanyents a **lligues de major nivell competitiu europeu**, especialment Espanya, Anglaterra, Itàlia i Alemanya.
- Els **estadis més emblemàtics** concentren un nombre més elevat de finals i registren, de mitjana, una **assistència superior**, evidenciant la relació entre infraestructura, prestigi i capacitat d’atracció d’espectadors.

En conjunt, el projecte demostra com la visualització de dades pot ser una eina poderosa per analitzar fenòmens esportius complexos, facilitant la identificació de patrons històrics i oferint una comprensió més profunda de l’evolució de la UEFA Champions League.